# Description

This notebook contains tools for debugging an individual `CcxtBroker` execution.

The notebook is responsible for aspects of the execution experiment run that are related to the correctness of the execution, as opposed to the analysis of the result:
- Consistency of logging
- Correctness in the submission of orders
- Consistency between trades and submitted orders
- Order time profiling

In [1]:
%load_ext autoreload
%autoreload 2

import logging

import pandas as pd

import helpers.hdbg as hdbg
import helpers.henv as henv
import helpers.hprint as hprint
import oms.ccxt.ccxt_filled_orders_reader as occforre

/app/core/signal_processing/decorrelation.py:12: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm


In [2]:
hdbg.init_logger(verbosity=logging.INFO)

_LOG = logging.getLogger(__name__)

_LOG.info("%s", henv.get_system_signature()[0])

hprint.config_notebook()

INFO: > cmd='/venv/lib/python3.8/site-packages/ipykernel_launcher.py -f /home/.local/share/jupyter/runtime/kernel-93e37837-7096-49d6-a87d-073df6b484e1.json'
-----------------------------------------------------------------------------
This code is not in sync with the container:
code_version='1.4.3' != container_version='1.4.0'
-----------------------------------------------------------------------------
You need to:
- merge origin/master into your branch with `invoke git_merge_master`
- pull the latest container with `invoke docker_pull`
INFO  # Git
  branch_name='CmTask4626_update_time_stats_logging'
  hash='138cc2f38'
  # Last commits:
    * 138cc2f38 tamriq   CmTask4626: replace get_machine_wall_clock_time with self.market_data.get_wall_clock_time() (  17 hours ago) Mon Jun 26 21:29:05 2023  (HEAD -> CmTask4626_update_time_stats_logging, origin/CmTask4626_update_time_stats_logging)
    * 39dbb786e Nina Lee CmTask4487_set_abort_on_error_to_False_in_reconcile_create_dirs (#4611) (  2

In [3]:
# Here provide a directory containing experiment results.
log_dir = "/app/system_log_dir/"
ccxt_log_reader = occforre.CcxtLogsReader(log_dir)

In [4]:
data = ccxt_log_reader.load_all_data()
ccxt_order_response_df = data["ccxt_order_responses"]
ccxt_trades_df = data["ccxt_trades"]
child_order_df = data["oms_child_orders"]

Loading files from '/app/rt_data/system_log_dir_20230627-063500/ccxt_child_order_responses':   0%|          | …

Loading files from '/app/rt_data/system_log_dir_20230627-063500/child_order_fills/ccxt_trades':   0%|         …

Loading files from '/app/rt_data/system_log_dir_20230627-063500/oms_child_orders':   0%|          | 0/10 [00:0…

## Child order responses

In [5]:
ccxt_order_response_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10 entries, 0 to 9
Data columns (total 25 columns):
 #   Column                  Non-Null Count  Dtype              
---  ------                  --------------  -----              
 0   info                    10 non-null     object             
 1   order                   10 non-null     object             
 2   client_order_id         10 non-null     object             
 3   timestamp               5 non-null      float64            
 4   datetime                5 non-null      object             
 5   last_trade_timestamp    0 non-null      object             
 6   symbol                  10 non-null     object             
 7   order_type              10 non-null     object             
 8   time_in_force           10 non-null     object             
 9   post_only               10 non-null     bool               
 10  reduce_only             10 non-null     bool               
 11  side                    10 non-null     object  

## Child orders

In [6]:
child_order_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 10 entries, 0 to 9
Data columns (total 19 columns):
 #   Column              Non-Null Count  Dtype              
---  ------              --------------  -----              
 0   creation_timestamp  10 non-null     datetime64[ns, UTC]
 1   asset_id            10 non-null     int64              
 2   type_               10 non-null     object             
 3   start_timestamp     10 non-null     datetime64[ns, UTC]
 4   end_timestamp       10 non-null     datetime64[ns, UTC]
 5   curr_num_shares     10 non-null     float64            
 6   diff_num_shares     10 non-null     float64            
 7   tz                  10 non-null     object             
 8   extra_params        10 non-null     object             
 9   passivity_factor    10 non-null     float64            
 10  latest_bid_price    10 non-null     float64            
 11  latest_ask_price    10 non-null     float64            
 12  bid_price_mean      10 non-null     flo

In [7]:
# Convert ccxt_id to string to avoid type confusion.
child_order_df["ccxt_id"] = child_order_df["ccxt_id"].astype(str)

In [8]:
child_order_df["extra_params"][0]

{'stats': {'child_order_created': Timestamp('2023-06-27 09:31:48.943538-0400', tz='America/New_York'),
  'child_order_limit_price_calculated': Timestamp('2023-06-27 09:31:48.948011-0400', tz='America/New_York'),
  'child_order_submitted': Timestamp('2023-06-27 09:31:49.459447-0400', tz='America/New_York'),
  'aligned_with_next_child_order_start_timestamp': Timestamp('2023-06-27 09:32:49.520546-0400', tz='America/New_York')},
 'max_leverage': 50,
 'submit_single_order_to_ccxt.start.timestamp': Timestamp('2023-06-27 09:31:48.949281-0400', tz='America/New_York'),
 'ccxt_id': 30183997106,
 'submit_single_order_to_ccxt.num_attempts': 0,
 'submit_single_order_to_ccxt.end.timestamp': Timestamp('2023-06-27 09:31:49.459322-0400', tz='America/New_York')}

## Trades

In [9]:
ccxt_trades_df.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 6 entries, 2023-06-27 13:31:49.844000+00:00 to 2023-06-27 13:36:18.526000+00:00
Data columns (total 22 columns):
 #   Column            Non-Null Count  Dtype              
---  ------            --------------  -----              
 0   timestamp         6 non-null      datetime64[ns, UTC]
 1   datetime          6 non-null      datetime64[ns, UTC]
 2   symbol            6 non-null      object             
 3   asset_id          6 non-null      int64              
 4   id                6 non-null      object             
 5   order             6 non-null      object             
 6   side              6 non-null      object             
 7   takerOrMaker      6 non-null      object             
 8   price             6 non-null      float64            
 9   amount            6 non-null      float64            
 10  cost              6 non-null      float64            
 11  transaction_cost  6 non-null      float64            
 12  fee

### Verify that all trades correspond to logged child orders

In [10]:
# Verify that all trades have an associated order response.
# If the trade does not have an associated order, it means
# that a trade was executed that was not accounted for during the experiment.
ccxt_trades_df["order"].isin(ccxt_order_response_df["order"])

timestamp
2023-06-27 13:31:49.844000+00:00    True
2023-06-27 13:32:54.573000+00:00    True
2023-06-27 13:33:58.636000+00:00    True
2023-06-27 13:35:03.299000+00:00    True
2023-06-27 13:36:07.907000+00:00    True
2023-06-27 13:36:18.526000+00:00    True
Name: order, dtype: bool

In [11]:
# Verify that all trades have an associated 'ccxt_id' in OMS child orders.
# This verifies that there is a consistency between a CCXT order, OMS order and
# a CCXT trade. If a trade has no associated OMS child order `ccxt_id`, it means
# that an unexpected trade was executed, for example, by a different actor on the same account.
ccxt_trades_df["order"].isin(child_order_df["ccxt_id"])

timestamp
2023-06-27 13:31:49.844000+00:00    True
2023-06-27 13:32:54.573000+00:00    True
2023-06-27 13:33:58.636000+00:00    True
2023-06-27 13:35:03.299000+00:00    True
2023-06-27 13:36:07.907000+00:00    True
2023-06-27 13:36:18.526000+00:00    True
Name: order, dtype: bool

### Check orders that do not correspond to any trades

In [12]:
# Existence of such orders is not necessarily a bug.
# It means that a given OMS child order was not filled.
child_orders_with_no_trades = ~child_order_df["ccxt_id"].isin(
    ccxt_trades_df["order"]
)
child_orders_with_no_trades.sum()

4

In [13]:
child_order_df.loc[child_orders_with_no_trades]

,creation_timestamp,asset_id,type_,start_timestamp,end_timestamp,curr_num_shares,diff_num_shares,tz,extra_params,passivity_factor,latest_bid_price,latest_ask_price,bid_price_mean,ask_price_mean,used_bid_price,used_ask_price,limit_price,ccxt_id,name
order_id,,,,,,,,,,,,,,,,,,,
0,2023-06-27 13:31:48.942768+00:00,3065029174,limit,2023-06-27 13:31:48.942768+00:00,2023-06-27 13:32:48.942768+00:00,-900.0,-225.0,America/New_York,{'stats': {'child_order_created': 2023-06-27 0...,0.55,0.06531,0.06532,0.065285,0.065295,latest_bid_price,latest_ask_price,0.065315,30183997106,0
2,2023-06-27 13:32:53.671325+00:00,3065029174,limit,2023-06-27 13:32:53.671325+00:00,2023-06-27 13:33:53.671325+00:00,-900.0,-225.0,America/New_York,{'stats': {'child_order_created': 2023-06-27 0...,0.55,0.06525,0.06526,0.065250,0.065260,latest_bid_price,latest_ask_price,0.065255,30184012154,2
5,2023-06-27 13:33:58.761872+00:00,6051632686,limit,2023-06-27 13:33:58.761872+00:00,2023-06-27 13:34:58.761872+00:00,88.0,8.0,America/New_York,{'stats': {'child_order_created': 2023-06-27 0...,0.55,2.32600,2.32700,2.325850,2.326850,latest_bid_price,latest_ask_price,2.326450,9195908368,5
7,2023-06-27 13:35:03.427969+00:00,6051632686,limit,2023-06-27 13:35:03.427969+00:00,2023-06-27 13:36:03.427969+00:00,88.0,8.0,America/New_York,{'stats': {'child_order_created': 2023-06-27 0...,0.55,2.33100,2.33200,2.330167,2.331167,latest_bid_price,latest_ask_price,2.331450,9195926707,7


### Check the correctness of trade amount
- Sum of `amount` of all trades related to a single child order should not exceed the total amount for that order.


In [14]:
# If the traded amount is larger than the order, it means that an extra trade
# is executed and the order is filled for a larger amount.
# This can mean an accounting error on the exchange side.
trade_amount_by_order = ccxt_trades_df.groupby("order").agg({"amount": sum})
trade_amount_by_order["child_order_amount"] = child_order_df.set_index("ccxt_id")[
    "diff_num_shares"
].abs()

In [15]:
mask = (
    trade_amount_by_order["amount"] > trade_amount_by_order["child_order_amount"]
)
trade_amount_by_order.loc[mask]

,amount,child_order_amount
order,,


### Verify that the number of child order responses equals number of child orders

In [16]:
# If the check is not passed, it means that some submitted child orders
# never reached the exchange and were not posted.
len(child_order_df) == len(ccxt_order_response_df)

True

### Verify that CCXT IDs are equal in both child orders and responses

In [17]:
# Verify that we are not logging any CCXT orders that are not a part
# of the experiment, i.e. have no OMS order associated with it.
set(child_order_df["ccxt_id"].unique()) == set(
    ccxt_order_response_df["order"].unique()
)

True

# Time profiling for child orders

In [18]:
# TODO(Toma): Update timestamp logging to use datetime.
child_order_df_with_timestamps = ccxt_log_reader.load_oms_child_order_df(
    unpack_extra_params=True
)

Loading files from '/app/rt_data/system_log_dir_20230627-063500/oms_child_orders':   0%|          | 0/10 [00:0…

In [19]:
child_order_timestamp_stats = child_order_df_with_timestamps["stats"].apply(
    pd.Series
)

In [20]:
child_order_timestamp_stats

,child_order_created,child_order_limit_price_calculated,child_order_submitted,aligned_with_next_child_order_start_timestamp
order_id,,,,
0,2023-06-27 09:31:48.943538-04:00,2023-06-27 09:31:48.948011-04:00,2023-06-27 09:31:49.459447-04:00,2023-06-27 09:32:49.520546-04:00
2,2023-06-27 09:32:53.672166-04:00,2023-06-27 09:32:53.676161-04:00,2023-06-27 09:32:54.186297-04:00,2023-06-27 09:33:54.246523-04:00
4,2023-06-27 09:33:58.250499-04:00,2023-06-27 09:33:58.254364-04:00,2023-06-27 09:33:58.760909-04:00,2023-06-27 09:34:58.818402-04:00
6,2023-06-27 09:35:02.912555-04:00,2023-06-27 09:35:02.916453-04:00,2023-06-27 09:35:03.426940-04:00,2023-06-27 09:36:03.488288-04:00
8,2023-06-27 09:36:07.520821-04:00,2023-06-27 09:36:07.524843-04:00,2023-06-27 09:36:08.030331-04:00,2023-06-27 09:37:08.091474-04:00
1,2023-06-27 09:31:49.461225-04:00,2023-06-27 09:31:49.465140-04:00,2023-06-27 09:31:49.972919-04:00,2023-06-27 09:32:49.975246-04:00
3,2023-06-27 09:32:54.188200-04:00,2023-06-27 09:32:54.191886-04:00,2023-06-27 09:32:54.698268-04:00,2023-06-27 09:33:54.700295-04:00
5,2023-06-27 09:33:58.762741-04:00,2023-06-27 09:33:58.766393-04:00,2023-06-27 09:33:59.274797-04:00,2023-06-27 09:34:59.277014-04:00
7,2023-06-27 09:35:03.428803-04:00,2023-06-27 09:35:03.432626-04:00,2023-06-27 09:35:03.942839-04:00,2023-06-27 09:36:03.944777-04:00
